In [6]:
from sklearn.preprocessing import scale
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.svm import SVR, SVC
from sklearn.model_selection import cross_val_predict, cross_validate, KFold, StratifiedKFold

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [39]:
def splitXY(dfXY):
    lbls = ['ReactorType', 'CoolingTime', 'Enrichment', 'Burnup', 'OrigenReactor']
    dfX = dfXY.drop(lbls, axis=1)
    if 'total' in dfX.columns:
        dfX.drop('total', axis=1, inplace=True)
    r_dfY = dfXY.loc[:, lbls[0]]
    c_dfY = dfXY.loc[:, lbls[1]]
    e_dfY = dfXY.loc[:, lbls[2]]
    b_dfY = dfXY.loc[:, lbls[3]]
    return dfX, r_dfY, c_dfY, e_dfY, b_dfY

CV = 5
trainset = '../pkl_trainsets/2jul2018/22jul2018_trainset3_nucs_fissact_not-scaled.pkl'
trainXY = pd.read_pickle(trainset)
trainXY = trainXY.sample(frac=0.6)
trainX, rY, cY, eY, bY = splitXY(trainXY)
trainX = scale(trainX)

In [50]:
nonzeros0 = np.count_nonzero(trainX)
mbyn = np.prod(trainX.shape)
frac = 0.1
to_zero = (frac * mbyn).astype(int)
sparser = trainX

def find_nearest_n(array, value, n):
    array = np.asarray(array)
    X = np.abs(array - value)
    idx = np.argpartition(X, n)#X.argsort()[:n] #np.argpartition(X, n)[:n]
    print(idx.shape)
    #idx = np.where(X == X.min())
    #x = array[idx[0], idx[1]]
    return idx#, x

idx = find_nearest_n(sparser, 0, to_zero)
sparser[idx] = 0

nonzeros1 = np.count_nonzero(sparser)
print (to_zero, nonzeros0, nonzeros1, mbyn)

ValueError: kth(=69921) out of bounds (47)

In [44]:
def make_sparse(df, frac):
    

SyntaxError: unexpected EOF while parsing (<ipython-input-44-9f7009bd7e2c>, line 2)

In [ ]:
# loops through each reactor parameter to do separate predictions
for Y in ('b', 'r', 'e', 'c'):
    trainY = pd.Series()
    # get param names and set ground truth
    if Y == 'c':
        trainY = cY
        parameter = 'cooling'
        k = 3 #7
        depth = 50 #50, 12
        feats = 25 #36, 47 
        g = 0.06 #0.2
        c = 50000 #200, 75000
    elif Y == 'e': 
        trainY = eY
        parameter = 'enrichment'
        k = 7 #8
        depth = 50 #53, 38
        feats = 25 #33, 16 
        g = 0.8 #0.2
        c = 25000 #420
    elif Y == 'b':
        # burnup needs much less training data...this is 24% of data set
        trainXY = trainXY.sample(frac=0.4)
        trainX, rY, cY, eY, bY = splitXY(trainXY)
        trainX = scale(trainX)
        trainY = bY
        parameter = 'burnup'
        k = 7 #4
        depth = 50 #50, 78
        feats = 25 #23, 42 
        g = 0.25 #0.025
        c = 42000 #105
    else:
        trainY = rY
        parameter = 'reactor'
        k = 3 #1, 2, or 12
        depth = 50 #50, 97
        feats = 25 # 37, 37 
        g = 0.07 #0.2
        c = 1000 #220
        
    csv_name = 'trainset3_fissact_m60_' + parameter
        
    ## initialize learners
    score = 'explained_variance'
    kfold = KFold(n_splits=CV, shuffle=True)
    knn_init = KNeighborsRegressor(n_neighbors=k, weights='distance')
    dtr_init = DecisionTreeRegressor(max_depth=depth, max_features=feats)
    svr_init = SVR(gamma=g, C=c)
    if Y is 'r':
        score = 'accuracy'
        kfold = StratifiedKFold(n_splits=CV, shuffle=True)
        knn_init = KNeighborsClassifier(n_neighbors=k, weights='distance')
        dtr_init = DecisionTreeClassifier(max_depth=depth, max_features=feats, class_weight='balanced')
        svr_init = SVC(gamma=g, C=c, class_weight='balanced')
    
    